In [3]:
import os
import mlflow
from dotenv import load_dotenv
load_dotenv()

# определяем основные credentials, которые нужны для подключения к MLflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные # поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# название эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = "bvv_43"
RUN_NAME = "log_model"

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 

In [4]:
import pandas as pd
import joblib
#from sklearn.compose import ColumnTransformer
#from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import StandardScaler, OneHotEncoder
#from catboost import CatBoostClassifier
#pip show category_encoders #2.6.3
#pip show scikit-learn #1.4

# прочитаем данные и обученную модель из 1 спринта
data = pd.read_csv('data/initial_data.csv')    
with open('models/fitted_model.pkl', 'rb') as fd:
    model = joblib.load(fd)

In [5]:
from sklearn.metrics import auc, precision_score, recall_score, f1_score,log_loss, confusion_matrix

val = data.head(20)
y_test= val.target
prediction = model.predict(val)
probas = model.predict_proba(val)

# Матрица ошибок (или confusion matrix)
#                    	Предсказан класс 0	Предсказан класс 1
# Фактически класс 0	True Negative (TN)	False Positive (FP) 
# Фактически класс 1	False Negative (FN)	True Positive (TP)
# Ошибка первого рода (False Positive, FP) - err1 
# Ошибка второго рода (False Negative — FN) - err2
confusion_matrix_data = confusion_matrix(y_test, prediction, normalize='all')
_, err1, _, err2 = confusion_matrix_data.ravel()
print(prediction, probas)
print (err1, err2)
print(confusion_matrix_data)

[0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 0 0 1 1 1] [[0.91319341 0.08680659]
 [0.25716922 0.74283078]
 [0.9465609  0.0534391 ]
 [0.02394008 0.97605992]
 [0.05253896 0.94746104]
 [0.3733327  0.6266673 ]
 [0.40603303 0.59396697]
 [0.15075113 0.84924887]
 [0.87996671 0.12003329]
 [0.64703817 0.35296183]
 [0.98295164 0.01704836]
 [0.64188079 0.35811921]
 [0.33222729 0.66777271]
 [0.27279345 0.72720655]
 [0.94561253 0.05438747]
 [0.95772252 0.04227748]
 [0.83146972 0.16853028]
 [0.47395417 0.52604583]
 [0.32721864 0.67278136]
 [0.00472072 0.99527928]]
0.2 0.35
[[0.45 0.2 ]
 [0.   0.35]]


In [6]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score,log_loss, confusion_matrix
# словарь со всеми метриками
metrics = {}

# посчитайте метрики из модуля sklearn.metrics
# err_1 — ошибка первого рода
# err_2 — ошибка второго рода

confusion_matrix_data = confusion_matrix(y_test, prediction, normalize='all')
_, err1, _, err2 = confusion_matrix_data.ravel()
auc = roc_auc_score(y_test, probas[:,1])
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, prediction)

# запишите значения метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss
metrics

{'err1': 0.2,
 'err2': 0.35,
 'auc': 0.9340659340659341,
 'precision': 0.6363636363636364,
 'recall': 1.0,
 'f1': 0.7777777777777778,
 'logloss': 7.20873067782343}

In [7]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('binary',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['paperless_billing',
                                                   'internet_service',
                                                   'online_security',
                                                   'online_backup',
                                                   'device_protection',
                                                   'tech_support',
                                                   'streaming_tv',
                                                   'streaming_movies', 'gender',
                                                   'partner', 'dependents',
                                                   'multiple_lines']),
                                                 ('cat',
                                                  CatBoostEncoder(return_df=False),
                                                  ['begin_date', 'type',
                                                   'payment_method']),
                                                 ('num', StandardScaler(),
                                                  ['end_date',
                                                   'monthly_charges',
                                                   'total_charges'])],
                                   verbose_feature_names_out=False)),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x7f448ae19c60>)])

##### Запись модели

In [9]:
EXPERIMENT_NAME = 'bvv_43'# ваш код здесь (напишите своё уникальное имя эксперимента)
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_bvv43"


# определяем основные credentials, которые нужны для подключения к MLflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
#os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("S3_ACCESS_KEY") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env
#os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("S3_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные # поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 


pip_requirements = "./requirements.txt"
#signature =  mlflow.models.infer_signature(X_test, prediction)
signature =  mlflow.models.infer_signature(data.head(6), model.predict(data.head(6)))
#input_example = X_test
input_example = data.head(6)
#metadata = metrics


# создаём эксперимент если его нет и записываем в него тестовую информацию
if not mlflow.get_experiment_by_name(EXPERIMENT_NAME):
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) 
else:
    experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    # ваш код здесь
    mlflow.log_metrics(metrics)
#    mlflow.log_metric("err1",metrics["err1"])
#    mlflow.log_metric("err2",metrics["err2"])
#    mlflow.log_metric("logloss",metrics["logloss"])
#    mlflow.log_metric("precision",metrics["precision"] )
#    mlflow.log_metric("recall",metrics["recall"] )
#    mlflow.log_metric("f1",metrics["f1"] )
#    mlflow.log_metric("auc",metrics["auc"] )

    model_info = mlflow.sklearn.log_model( 
#    model_info = mlflow.catboost.log_model( 
        artifact_path='models',
        sk_model=model,
        registered_model_name=REGISTRY_MODEL_NAME,
        pip_requirements=pip_requirements,
        signature=signature,
        input_example=input_example
        #await_registration_for=60
        #metadata=metrics
        #code_paths=code_paths
		)



/home/mle-user/mle_projects/mle-mlflow/.venv/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Registered model 'churn_model_bvv43' already exists. Creating a new version of this model...
2025/12/28 13:26:23 INFO mlflow.tracking._model_registry.c

<Experiment: artifact_location='s3://s3-student-mle-20251108-d3b615298f-freetrack/8', creation_time=1766832464070, experiment_id='8', last_update_time=1766832464070, lifecycle_stage='active', name='bvv_43', tags={}>

In [15]:
signature =  mlflow.models.infer_signature(pd.DataFrame([[1,2,3],[3,2,1],[2,3,4]]), pd.DataFrame([1,0,1]))
input_example = pd.DataFrame([[1,2,3],[3,2,1],[2,3,4]])

/home/mle-user/mle_projects/mle-mlflow/.venv/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
/home/mle-user/mle_projects/mle-mlflow/.venv/lib/python3.10/site-packages/mlflow/models/signature.py:213: UserWarning: Hint: Inferred schema contains i

##### Чтение модели

In [ ]:
client = mlflow.MlflowClient() 

REGISTRY_MODEL_NAME = "YP_MODEL"
models = client.search_model_versions(
    filter_string=f"name = '{REGISTRY_MODEL_NAME}'"
)

for model in models:
    print(model)

tag_key = "tags.'ab test'" # добавьте ключ тега, исходя из его названия в задании
tag_value = "' ab '"  # добавьте значение тега, исходя из его названия в задании
filter_string = f"{tag_key} ILIKE {tag_value}" # сформируйте строку для фильтрации модели
results = client.search_model_versions(filter_string)

model_name_1 = models[-1].name
model_version_1 = models[-1].version
model_stage_1 = models[-1].current_stage

model_name_2 = models[-2].name
model_version_2 = models[-2].version
model_stage_2 = models[-2].current_stage

print(f"Текущий stage последней модели: {model_stage_1}")
print(f"Текущий stage предпоследней модели 2: {model_stage_2}")

# Устанавлмваем статусы
client.transition_model_version_stage(model_name_1, model_version_1, "production")
client.transition_model_version_stage(model_name_2, model_version_2, "staging") 

client = mlflow.MlflowClient() 
#REGISTRY_MODEL_NAME = "churn_model_bvv43"

name_filter = "run_id" # напишите имя фильтра
model_num = "'7abcdf3786d942d184f19f37a92d2ea9'" # напишите номер модели
filter_string = f"{name_filter} = {model_num}" # сформируйте строку для фильтрации модели
models = client.search_model_versions(filter_string)
model_name = models[-1].name
model_version = models[-1].version
model_stage = models[-1].current_stage

print(f"Текущий stage модели {model_name} версии {model_version} : {model_stage}")



{'err1': 0.2,
 'err2': 0.35,
 'auc': 0.9340659340659341,
 'precision': 0.6363636363636364,
 'recall': 1.0,
 'f1': 0.7777777777777778,
 'logloss': 7.20873067782343}

In [ ]:
REGISTRY_MODEL_NAME = "churn_model_bvv43"

name_filter = "name" # напишите имя фильтра
model_num = 3 # напишите номер модели

filter_string = f"{name_filter} = '{REGISTRY_MODEL_NAME}'"

models = client.search_model_versions(filter_string)


model_version = models[model_num-1].version
model_stage = models[model_num-1].current_stage

print(f"Текущая версия модели {REGISTRY_MODEL_NAME}: {model_version}")
print(f"Текущая стадия модели {REGISTRY_MODEL_NAME}: {model_stage}")

In [13]:
client = mlflow.MlflowClient() 
REGISTRY_MODEL_NAME = "churn_model_bvv43"

name_filter = "name" # напишите имя фильтра
#model_num = 3 # напишите номер модели
filter_string = f"{name_filter} = '{REGISTRY_MODEL_NAME}'" # сформируйте строку для фильтрации модели
models = client.search_model_versions(filter_string)
model_name = models[-1].name
model_version = models[-1].version
model_stage = models[-1].current_stage

print(f"Текущий stage модели {model_name} версии {model_version} : {model_stage}")


Текущий stage модели churn_model_bvv43 версии 3 : Archived


In [ ]:
# Устанавливаем статусы
client.transition_model_version_stage(model_name_1, model_version_1, "production")
client.transition_model_version_stage(model_name_2, model_version_2, "staging") 

REGISTRY_MODEL_NAME = "YP_MODEL"

# Переименовываем модель
client.rename_registered_model(
    name=REGISTRY_MODEL_NAME, 
    new_name=f"{REGISTRY_MODEL_NAME}_new"
) 


[<ModelVersion: aliases=[], creation_timestamp=1766928383777, current_stage='None', description='', last_updated_timestamp=1766928383777, name='churn_model_bvv43', run_id='56f312fdbf7844cba41574824f3e4566', run_link='', source='s3://s3-student-mle-20251108-d3b615298f-freetrack/8/56f312fdbf7844cba41574824f3e4566/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='5'>,
 <ModelVersion: aliases=[], creation_timestamp=1766864171602, current_stage='None', description='', last_updated_timestamp=1766864171602, name='churn_model_bvv43', run_id='62a3bdb7f0814789ba62eb05d04ae646', run_link='', source='s3://s3-student-mle-20251108-d3b615298f-freetrack/8/62a3bdb7f0814789ba62eb05d04ae646/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='4'>,
 <ModelVersion: aliases=[], creation_timestamp=1766855099921, current_stage='Production', description='', last_updated_timestamp=1766856114131, name='churn_model_bvv43', run_id='8a011534674f499792

In [9]:
mlflow.get_experiment_by_name(EXPERIMENT_NAME)

<Experiment: artifact_location='s3://s3-student-mle-20251108-d3b615298f-freetrack/8', creation_time=1766832464070, experiment_id='8', last_update_time=1766832464070, lifecycle_stage='active', name='bvv_43', tags={}>